In [1]:
# wow, very many playgrounds
# this one is to just strictly work on optimization

In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import editdistance
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')
sys.path.append('/usr/local/ViennaRNA/lib/python3.7/site-packages/')
%reload_ext autoreload
%autoreload 2
# from utils.sequence_utils import *
from models.Theoretical_models import *
from models.Noise_wrapper import *
from exploration_strategies.CE import *
# from exploration_strategies.RL_explorers_modify import *
from utils.landscape_utils import *
from models.RNA_landscapes import *
from models.Multi_dimensional_model import *
import RNA

In [3]:
import torch
from torch import nn
from tqdm import tqdm_notebook as tqdm
import torch.nn.functional as F

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
RAA="UGCA" #alphabet
length=40
wt=generate_random_sequences(length,1,alphabet=RAA)[0]
print(wt)
#make a simple folding landscape starting at wt
landscape1=RNA_landscape(wt)
noise_alpha=1
virtual_per_measure_ratio=15
temperature=0.1
# there are multiple abstract "noise models" you can use, or you can try to train your own model, using skM
noisy_landscape_CE=Noise_wrapper(landscape1,noise_alpha=noise_alpha)
noisy_landscape_RL=Noise_wrapper(landscape1,noise_alpha=noise_alpha)
noisy_landscape_RL_multiple=Noise_wrapper(landscape1,noise_alpha=noise_alpha)
#noisy_landscape=Gaussian_noise_landscape(base_landscape,noise_alpha=0.15)
#noisy_landscape=DF_noise_landscape(base_landscape,noise_alpha=0.5)
batch_size = 10
initial_genotypes=list(set([wt]+[generate_random_mutant(wt,0.05,RAA) for i in range(batch_size*10)]))[:batch_size]
len(initial_genotypes)

ACUCAGUACAGAACGGUGGGGGUAUCUUAUACGACCAUAA


10

# SINGLE AGENT RL

In [5]:
import numpy as np
import random
import bisect
from utils.sequence_utils import translate_string_to_one_hot


def renormalize_moves(one_hot_input,rewards_output):
    """ensures that staying in place gives no reward"""
    zero_current_state=(one_hot_input-1)*-1
    return np.multiply(rewards_output,zero_current_state)

def walk_away_renormalize_moves(one_hot_input, one_hot_wt, rewards_output):
    """ensures that moving toward wt is also not useful"""
    zero_current_state=(one_hot_input-1)*-1
    zero_wt=((one_hot_wt-1)*-1)
    zero_conservative_moves=np.multiply(zero_wt,zero_current_state)
    return np.multiply(rewards_output,zero_conservative_moves)

def get_all_singles_fitness(model,sequence,alphabet):
    prob_singles=np.zeros((len(alphabet),len(sequence)))
    for i in range(len(sequence)):
        for j in range(len(alphabet)):
            putative_seq=sequence[:i]+alphabet[j]+sequence[i+1:]
           # print (putative_seq)
            prob_singles[j][i]=model.get_fitness(putative_seq)
    return prob_singles

def sample_greedy(matrix):
    i,j=matrix.shape
    max_arg=np.argmax(matrix)
    y=max_arg%j
    x=int(max_arg/j)
    output=np.zeros((i,j))
    output[x][y]=matrix[x][y]
    return output

def sample_multi_greedy(matrix):
    n = 5 # the number of base positions to greedily change
    max_args = np.argpartition(matrix.flatten(), -n)[-n:]
    i,j=matrix.shape
    output=np.zeros((i,j))
    for max_arg in max_args:
        y=max_arg%j
        x=int(max_arg/j)
        output[x][y]=matrix[x][y]
    return output

def sample_random(matrix):
    i,j=matrix.shape
    non_zero_moves=np.nonzero(matrix)
   # print (non_zero_moves)
    k=len(non_zero_moves)
    l=len(non_zero_moves[0])
    if k!=0 and l!=0:
        rand_arg=random.choice([[non_zero_moves[alph][pos] for alph in range(k)] for pos in range(l)])
    else:
        rand_arg=[random.randint(0,i-1),random.randint(0,j-1)]
    #print (rand_arg)
    y=rand_arg[1]
    x=rand_arg[0]
    output=np.zeros((i,j))
    output[x][y]=matrix[x][y]
    return output   
    
def construct_mutant_from_sample(pwm_sample, one_hot_base):
    one_hot=np.zeros(one_hot_base.shape)
    one_hot+=one_hot_base
    nonzero = np.nonzero(pwm_sample)
    nonzero = list(zip(nonzero[0], nonzero[1]))
    for nz in nonzero: # this can be problematic for non-positive fitnesses
        i, j = nz
        one_hot[:,j]=0
        one_hot[i,j]=1
    return one_hot

def make_one_hot_train_test(genotypes,model,alphabet,split=0.0):
    genotypes_one_hot=[translate_string_to_one_hot(genotype,alphabet) for genotype in genotypes]
    genotype_fitnesses=[get_all_singles_fitness(model,genotype,alphabet) for genotype in genotypes]
    train_x=[]
    test_x=[]
    train_y=[]
    test_y=[]
    if split>0: #check this to avoid calling the randomizer if not required
        for x,y in zip(genotypes_one_hot,genotype_fitnesses):
                if random.random()<split:
                    test_x.append(x)
                    test_y.append(y)
                else:
                    train_x.append(x)
                    train_y.append(y)
        train_x=np.stack(train_x)
        train_y=np.stack(train_y)
        test_y=np.stack(test_y)
        test_x=np.stack(test_x)
        return train_x,train_y,test_x,test_y
    else:
        train_x=np.stack(genotypes_one_hot)
        train_y=np.stack(genotype_fitnesses)

        return train_x,train_y,test_x,test_y

In [8]:
from collections import deque
# from utils.RL_utils import *
from utils.sequence_utils import translate_one_hot_to_string,generate_random_mutant
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


class RL_agent_QN():

    def __init__(self,start_sequence,alphabet,gamma=0.9,memory_size=1000, device = "cpu"):
        self.alphabet=alphabet
        self.state=translate_string_to_one_hot(start_sequence,self.alphabet)
        self.seq_size=len(start_sequence)
        self.actor=build_model(self.seq_size,len(self.alphabet), device)
        self.start_sequence=translate_string_to_one_hot(start_sequence,self.alphabet)
        self.gamma=gamma
        self.memory=[]
        self.memory_size=memory_size

    def reset_position(self,sequence):
        self.state=translate_string_to_one_hot(sequence,self.alphabet)

    def get_position(self):
        return translate_one_hot_to_string(self.state,self.alphabet)

    def translate_pwm_to_sequence(self,input_seq_one_hot,output_pwm):
        diff=output_pwm-input_seq_one_hot
        most_likely=np.argmax(diff,axis=0)
        out_seq=""
        for m in most_likely:
            out_seq+=self.alphabet[m]
        return out_seq

    def observe(self,landscape,genotypes,costly=False,num_observations=50,epsilon=0.2,depth=10):
        if genotypes:
            new_sequences=make_one_hot_genotypes(genotypes,self.alphabet)
        elif self.memory:
            new_sequences=make_one_hot_genotypes(self.memory[-num_observations:],self.alphabet)
        else:
            return "Nothing to observe"
        new_moves=[]
        all_moves=[]
        
        #all_fitnesses=[]
        for i in range(depth):
            for genotype in new_sequences:
                if random.random()<epsilon:
                    genotype_tensor = torch.from_numpy(np.expand_dims(genotype,axis=0)).float()
                    prediction = self.actor(genotype_tensor)
                    prediction = prediction.detach().numpy()[0]
                    #moves=walk_away_renormalize_moves(genotype,self.start_sequence,prediction)
                    moves=renormalize_moves(genotype,prediction)

                    sample=sample_random(moves)
                    mutant=construct_mutant_from_sample(sample,genotype)
                    new_moves.append(translate_one_hot_to_string(mutant,self.alphabet))
                else:
                    genotype_tensor = torch.from_numpy(np.expand_dims(genotype,axis=0)).float()
                    prediction = self.actor(genotype_tensor)
                    prediction = prediction.detach().numpy()[0]
                    #moves=walk_away_renormalize_moves(genotype,self.start_sequence,prediction)
                    moves=renormalize_moves(genotype,prediction)
                    
                    sample=sample_boltzman(moves)
                    mutant=construct_mutant_from_sample(sample,genotype)
                    new_moves.append(translate_one_hot_to_string(mutant,self.alphabet))
            if costly:
                landscape.measure_true_landscape(new_moves)
#             else:
#                 landscape.genotype_fitnesses(new_moves)

            new_sequences=np.stack([translate_string_to_one_hot(move,self.alphabet) for move in new_moves])
            all_moves.extend(new_moves)
            #all_fitnesses.append(new_fitnesses)
            new_moves=[]
        self.memory=all_moves[-self.memory_size:] #store at most 1000 memories
        return all_moves

    def train_actor(self, landscape, observations, train_epochs=10):
        loss_fxn = nn.MSELoss()
        optimizer = optim.Adam(self.actor.parameters(), lr=0.001)
        total_loss = 0.
        for epoch in range(train_epochs):     
            moves = observations #random.sample(observations, train_size)
            genotypes_one_hot, genotype_fitnesses, t_1, t_2 = make_one_hot_train_test(moves, landscape, self.alphabet)
            optimizer.zero_grad()
            genotypes_one_hot_tensor = torch.from_numpy(genotypes_one_hot).float()
            outputs = self.actor(genotypes_one_hot_tensor)
            genotype_fitnesses = torch.tensor(genotype_fitnesses, requires_grad=True).float()
            loss = loss_fxn(outputs, genotype_fitnesses)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
#         print('[%d] Avg loss: %.3f' % (epoch+1, total_loss / (epoch + 1)))
        return (total_loss / (epoch+1))

    def xtrain_actor(self,landscape,observations,train_size=1000,train_epochs=1):
        loss_fxn = nn.MSELoss()
        optimizer = optim.Adam(net.parameters, lr=0.001)
        for epoch in range(train_epochs):  # loop over the dataset multiple times
            running_loss = 0.0
            
            moves = random.sample(observations,train_size)+random.sample(self.memory,train_size)
            genotypes_one_hot, genotype_fitnesses, t_1, t_2 = make_one_hot_train_test(moves, landscape, self.alphabet)
            
            optimizer.zero_grad()
            
            outputs = self.actor(genotypes_one_hot)
            loss = loss_fxn(outputs, genotype_fitnesses)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            if i % 2000 == 1999:    # print every 2000 mini-batches
                    print('[%d, %5d] loss: %.3f' %
                          (epoch + 1, i + 1, running_loss / 2000))
                    running_loss = 0.0

    def pick_action(self,method):
            state_tensor = torch.from_numpy(np.expand_dims(self.state,axis=0)).float()
            prediction = self.actor(state_tensor)
            prediction = prediction.detach().numpy()[0]
            moves=renormalize_moves(self.state,prediction)
            
            if(method[0] == 'eps'):
                epsilon = method[1]
                p = random.random()
                sample = sample_random(moves) if p < epsilon else sample_greedy(moves)
            elif(method[0] == 'multi_eps'):
                epsilon = method[1]
                p = random.random()
                sample = sample_random(moves) if p < epsilon else sample_multi_greedy(moves)
            elif(method[0] == 'rollout'):
                epsilon = method[1]
                p = random.random()
                if p < epsilon:
                    sample = sample_random(moves)
                else:
                    n = 5
                    for i in range(n):
                        sample = sample_greedy(moves)
                        mutant = construct_mutant_from_sample(sample,self.state)
                        state_tensor = torch.from_numpy(np.expand_dims(mutant,axis=0)).float()
                        prediction = self.actor(state_tensor)
                        prediction = prediction.detach().numpy()[0]
                        moves = renormalize_moves(mutant,prediction)
                    sample = sample_greedy(moves)
                    
            mutant=construct_mutant_from_sample(sample,self.state)
            mutant_string=translate_one_hot_to_string(mutant,self.alphabet)

            self.state=mutant
            self.memory.append(mutant_string)

            return mutant_string
        
class Model(nn.Module):
    def __init__(self, sequence_len, alphabet_len):
        super(Model,self).__init__()
        self.sequence_len = sequence_len
        self.alphabet_len = alphabet_len
        self.linear1 = nn.Linear(alphabet_len * sequence_len, alphabet_len * sequence_len)
        self.linear2 = nn.Linear(alphabet_len * sequence_len, sequence_len)
        self.linear3 = nn.Linear(sequence_len, sequence_len)
        self.linear4 = nn.Linear(sequence_len, 18)
        self.linear5 = nn.Linear(18, alphabet_len * sequence_len)
        self.linear6 = nn.Linear(alphabet_len * sequence_len, alphabet_len * sequence_len)
        self.linear_final = nn.Linear(sequence_len, sequence_len)
    
    def forward(self, x):
        x = x.view(-1, self.alphabet_len*self.sequence_len)
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = F.relu(self.linear3(x))
        x = F.relu(self.linear4(x))
        x = F.relu(self.linear5(x))
        x = F.relu(self.linear6(x))
        x = x.view([-1]+list((self.alphabet_len, self.sequence_len)))
        x = self.linear_final(x)
        return x
    
    
def build_model(sequence_len, alphabet_len, device):
    model = Model(sequence_len, alphabet_len).to(device)
    return model

In [9]:
def run_RL(initial_genotypes, batch_size, generations, sampling_method):
    noisy_landscape_RL.reset()
    noisy_landscape_RL.measure_true_landscape(initial_genotypes)
    noisy_landscape_RL.natural_mode=False
    noisy_landscape_RL.local_mode=False
    noisy_landscape_RL.cost

    agent = RL_agent_QN(wt,alphabet=RAA,memory_size=10000, device=device)
    observations=[x for x in initial_genotypes]#agent.observe(noisy_landscape_RL,initial_genotypes,costly=True,num_observations=100,depth=5)
    agent.train_actor(noisy_landscape_RL,observations, train_epochs=10)
    new_sequences = sorted([(noisy_landscape_RL.get_fitness(observations[i]),[observations[i]])
                            for i in range(len(observations))])

    RL_top_seqs = []
    while noisy_landscape_RL.cost<batch_size*generations:
        if (sampling_method == 'eps'):
            eps = max(0.2,(0.5 - noisy_landscape_RL.cost / (batch_size * generations)))
            method = (sampling_method, eps)
        elif (sampling_method == 'multi_eps'):
            eps = max(0.2,(0.5 - noisy_landscape_RL.cost / (batch_size * generations)))
            method = (sampling_method, eps)
        elif (sampling_method == 'rollout'):
            eps = max(0.2,(0.5 - noisy_landscape_RL.cost / (batch_size * generations)))
            method = (sampling_method, eps)
        else:
            method = (sampling_method,)
        
        b = 0
        new = []
        while(b < batch_size):
            s = agent.pick_action(method)
            if not s in noisy_landscape_RL.measured_sequences.keys():
                new += [s]
                b += 1
    #     new=[agent.pick_action_epsilon_greedy(epsilon=eps) for _ in range(batch_size)]
        noisy_landscape_RL.measure_true_landscape(new)
        observations += new
        avg_loss = agent.train_actor(noisy_landscape_RL,observations, train_epochs=20)
        print (noisy_landscape_RL.cost, noisy_landscape_RL.get_fitness(new_sequences[-1][1][0]), avg_loss)
        new_sequences += [(noisy_landscape_RL.get_fitness(new[i]),[new[i]]) for i in range(len(new))]
        new_sequences = sorted(new_sequences)

        RL_top_seqs += [(noisy_landscape_RL.cost, new_sequences[-1][0])]
    return RL_top_seqs

In [10]:
batch_size = 10
generations = 10
RL_data = run_RL(initial_genotypes, batch_size, generations, 'multi_eps')

18 0.10941176694982192 0.005418052885215729
28 0.10941176694982192 0.0019043981097638606
38 0.18117646610035615 0.0007166770636104047
47 0.18117646610035615 0.0007371568994130939
53 0.18117646610035615 0.0005877455972949974
59 0.19411764705882353 0.0010119560349266977
62 0.3105882308062385 0.0007589524320792407
66 0.39176469690659466 0.001146218180656433
72 0.44352942074046414 0.0007976353954290971
80 0.44352942074046414 0.0007265045889653266
89 0.44352942074046414 0.0008908253425033763
99 0.44352942074046414 0.0007590197856188752
102 0.4658823349896599 0.0008459628035780043


# MULTI AGENT RL

In [11]:
import random

class RL_multi_agent_QN():
    def __init__(self,agent_type,landscape,wt,alphabet, pop_size,memory_size=1200,best_seq_prop=1):
        self.memory=[]
        self.pop_size=pop_size
        self.alphabet=alphabet
        self.wt=wt
        self.agents=[agent_type(self.wt,self.alphabet) for i in range(self.pop_size)]
        self.memory_size=memory_size
        self.memory=[]
        self.landscape=landscape
        self.best_seqs=[]
        self.best_seqs_len=self.pop_size*best_seq_prop

    def add_to_top_seqs(self,sequence):
        fitness= self.landscape.get_fitness(sequence)
        if len(self.best_seqs)<self.best_seqs_len:
            self.best_seqs.append((fitness,sequence))

        else:
            if fitness > self.best_seqs[0][0] and (fitness,sequence) not in self.best_seqs:
                self.best_seqs.append((fitness,sequence))
                self.best_seqs=sorted(self.best_seqs)
                self.best_seqs=self.best_seqs[1:]

    def reset_position(self,sequence):
        for agent in self.agents:
            agent.reset_position(sequence)

    def observe(self, num_observations=1000, costly=False, depth=10):
        for agent in self.agents:
            observations=agent.observe(self.landscape,[generate_random_mutant(agent.get_position(),0.1,self.alphabet) for i in range(num_observations)],costly=True,depth=depth)
            self.memory.extend(observations)
        if len(self.memory)>self.memory_size:
            self.memory=random.sample(self.memory,self.memory_size)
            
    def force_observations(self, observations):
        num_agents = len(self.agents)
        for agent in self.agents:
            agent.memory.extend(observations)
            if len(agent.memory) > agent.memory_size:
                agent.memory = random.sample(agent.memory, agent.memory_size)

    def train_agents(self, observations):
        total_loss = 0.0
        for agent in self.agents:
            total_loss += agent.train_actor(self.landscape, observations, train_epochs=20)
        return total_loss/len(self.agents)

    def cross_train_agents(self, observations, num_x_obs=1000):
        total_loss = 0.0
        for agent in self.agents:
            total_loss += agent.xtrain_actor(self.landscape, random.sample(observations,num_x_obs), train_size=max(len(observations), num_x_obs), train_epochs=20)
        return total_loss/len(self.agents)

    def pick_action(self, method):
        self.sequences=[]
        sequences=[]
        for agent_id, agent in enumerate(self.agents):
            action = agent.pick_action(method)
            self.sequences.append(action)
            sequences.append((action, agent_id))
#             self.memory.extend(agent.memory)

#         if len(self.memory)>self.memory_size:
#             self.memory=random.sample(self.memory,self.memory_size)

        for seq in self.sequences:
            self.add_to_top_seqs(seq)

        return sequences

In [12]:
def run_RL_multi_agent(initial_genotypes, batch_size, generations, sampling_method, num_agents=10):
    noisy_landscape_RL_multiple.reset()
    noisy_landscape_RL_multiple.measure_true_landscape(initial_genotypes)
    noisy_landscape_RL_multiple.natural_mode=False
    noisy_landscape_RL_multiple.local_mode=False
    noisy_landscape_RL_multiple.cost

    multi_agents = RL_multi_agent_QN(RL_agent_QN,
                              noisy_landscape_RL_multiple,
                              wt,
                              RAA,
                              num_agents,
                              memory_size=batch_size)

    # set the SAME initial position for each
    for i in range(num_agents):
        multi_agents.agents[i].reset_position(initial_genotypes[0])

    # keep separate list of observations for each agent
    observations = [[x for x in initial_genotypes] for i in range(num_agents)]
    new_sequences = sorted([(noisy_landscape_RL_multiple.get_fitness(observations[i]),[observations[i]])
                            for i in range(len(observations))])

    multi_RL_top_seqs = []
    
    while noisy_landscape_RL_multiple.cost < batch_size * generations:
        if (sampling_method == 'eps'):
            eps = max(0.2,(0.5 - noisy_landscape_RL.cost / (batch_size * generations)))
            method = (sampling_method, eps)
        elif (sampling_method == 'multi_eps'):
            eps = max(0.2,(0.5 - noisy_landscape_RL.cost / (batch_size * generations)))
            method = (sampling_method, eps)
        elif (sampling_method == 'rollout'):
            eps = max(0.2,(0.5 - noisy_landscape_RL.cost / (batch_size * generations)))
            method = (sampling_method, eps)
        else:
            method = (sampling_method,)
        
        # DOESN'T WORK: goal; train each agent on their own observations, and a subsample of other agents observations
        b = 0
        new = []
        new_indices = []
        while(b < batch_size):
            seqs = multi_agents.pick_action(method)
            for s in seqs:
                if not s[0] in noisy_landscape_RL_multiple.measured_sequences.keys():
                    new += [s[0]]
                    new_indices += [s[1]]
                    b += 1

        noisy_landscape_RL_multiple.measure_true_landscape(new)
        observations += new
    #     multi_agents.force_observations(observations)
        avg_loss = multi_agents.train_agents(observations)
        print(noisy_landscape_RL_multiple.cost, noisy_landscape_RL_multiple.get_fitness(new_sequences[-1][1][0]), avg_loss)
        new_sequences += [(noisy_landscape_RL_multiple.get_fitness(new[i]),[new[i]]) for i in range(len(new))]
        new_sequences = sorted(new_sequences)
        multi_RL_top_seqs += [(noisy_landscape_RL_multiple.cost, new_sequences[-1][0])]
        
    return multi_RL_top_seqs

In [13]:
batch_size = 10
generations = 10
num_agents = 2
RL_data = run_RL_multi_agent(initial_genotypes, batch_size, generations, 'multi_eps', num_agents)

TypeError: unhashable type: 'list'